In [1]:
import sys
sys.path.append('../_amigocloud')

In [2]:
import collections
from amigocloud import AmigoCloud

from datetime import datetime

from docxtpl import DocxTemplate
import docxtpl
from docx.shared import Mm
import requests

import qrcode

In [3]:
# ruta de carpera donde estan las llaves
ruta = open('../ruta.txt')
ruta = ruta.readlines()[0]
# leer token
acceso = open(ruta + '/_keys/api_amigocloud.key','r')
api_token = acceso.readlines()[0]
acceso.close()
amigocloud = AmigoCloud(token=api_token)
amigocloud

In [4]:
# variables globales
id_proyecto = 34080

## convertir formato de fecha

In [5]:
# convierte de formato YYYY-mm-dd H:M:S+z a d/m/YYYY
def convertir_formato_fecha(fecha):
    new_formato = datetime.strptime(fecha, "%Y-%m-%d %H:%M:%S%z").strftime("%d/%m/%Y")
    return new_formato

## ejecutar query

In [6]:
# ejecuta cualquier query sql en el proyecto que se le indique
# requiere el id de proyecto, query a ejecutar y tipo solicitud (get o post)
def ejecutar_query_sql(id_project, query, tipo_sql):
    # define la url del proyecto para ejecutar el querry
    url_proyecto_sql = f'https://app.amigocloud.com/api/v1/projects/{id_project}/sql'
    # crea la estructura de query para amigocloud
    query_sql = {'query': query}
    
    resultado_get = ''
    # eleige que tipo de solicitud se realizara (get o post)
    if tipo_sql == 'get': 
        resultado_get = amigocloud.get(url_proyecto_sql, query_sql)
    elif tipo_sql == 'post':
        resultado_get = amigocloud.post(url_proyecto_sql, query_sql)
    else:
        resultado_get = 'Se a seleccionado un tipo de solicitud erroneo.'
    return resultado_get

## ejecuta query segun id_query (ejecucion de queryes de un proyecto)

In [7]:
# ejecuta un query que esta almacenado en un proyecto de amigocloud (generalmente un update),
# requiere id de proyecto e id de query
# retorna cuantas filas fueron afectadas
def ejecutar_query_por_id(id_project, id_query):
    # obtiene el query basado en el id_project y el id_query
    get_query = amigocloud.get(f'https://app.amigocloud.com/api/v1/projects/{id_project}/queries/{id_query}')
    # se extrae solo el texto del query
    query = get_query['query']
    # ejecuta el query_sql con metodo post y guarda la respuesta
    respuesta_post = ejecutar_query_sql(id_proyecto, query, 'post')
    # retorna el numero de filas afectadas por el query
    return respuesta_post['count']

## generar qr

In [8]:
# genera un qr con datos texto que le se envie
# requiere le texto de contenido, ruta donde se guardara el qr y nombre para guardar
def generar_qr(contenido_qr, ruta_save, id_name):
    # declara obj del qrocode
    qr = qrcode.QRCode(
        version=1,
        error_correction=qrcode.constants.ERROR_CORRECT_L,
        box_size=10,
        border=4)
    # se agrega el contenido del qr
    qr.add_data(contenido_qr)
    qr.make(fit=True)
    # define colores del qr
    img = qr.make_image(fill_color="black", back_color="white")
    # define ruta y nombre para guardar el qr
    ruta_qr = ruta_save + id_name + '.png'
    # guardar el código QR en un archivo .png
    img.save(ruta_qr)
    # retorna la ruta del qr
    return ruta_qr

## convertir un dict a obj

In [16]:
# convierte un dict a un obj
# recibe el dict y el nombre con el que se creara el obj
def convertir_dict_obj(diccionario, name):
    return collections.namedtuple(name, diccionario.keys())(*diccionario.values())

## buscar fotos

In [61]:
# busca las fotos en una galeria determinada, y que corresponda con un id
# recibe el nombre de una galeria y el id correspondiente
def buscar_fotos(galeria, source_id):
    # se construye el query
    query_sql = f"select * from {galeria} where source_amigo_id='{source_id}'"
    # se ejecuta el query con un metodo get
    respuesta = ejecutar_query_sql(id_proyecto, query_sql, 'get')
    # retorna un json completo de las fotos encontradas
    return respuesta

## descargar la lista de fotos y convertir a inline

In [85]:
# converir img en inline
# recibe el documento sobre el que se agrega el inline, la ruta de la imagen, y en ancho de la imagen para agregar al doc
def convertir_img_to_inline(docu, filename, ancho):
    return docxtpl.InlineImage(docu, image_descriptor = filename, width=Mm(ancho))

In [86]:
# descarga las fotos, las convierte en inline
# recibe la lista de fotos (json, con estructura de galeria de amigocloud), ruta donde guardar las fotos y documento sobre el que se agrega los inline
def descargar_convertir_fotos_inline(lista_fotos, ruta_save, documento):
    # crea lista vacia para acumular las fotos inline
    lista_fotos_inline = []
    # recorre la lista de fotos (json)
    for foto in lista_fotos:
        # posos para descargar img
        response = requests.get("https://www-amigocloud.s3.amazonaws.com/gallery/" + foto['s3_filename'])
        file_name = ruta_save + foto['s3_filename']
        file = open(file_name, "wb")
        file.write(response.content)
        file.close()
        # convertir imagen descargada a inline
        inline_img = convertir_img_to_inline(documento, file_name, 50)
        # agrega el inline a la lista
        lista_fotos_inline.append(inline_img)
    # retorna la lista inline
    return lista_fotos_inline

In [104]:
id_insp = 20

In [105]:
# consulta pata extraer los datos de inspeccion por id concreto
query = f'select ca.canhero, ca.maquina, ca.modelo, ca.codigo_maquina, ca.frente, insp.* from dataset_341171 insp\
    inner join dataset_341170 ca on ca.amigo_id=insp.maquina_ref_preven\
    where insp.id={id_insp}'
# ejecucion del query
select = ejecutar_query_sql(id_proyecto, query, 'get')
data = select['data'][0]

In [106]:
# modificacion de la fecha a formato dd/mm/aaaa
data['fecha_registro'] = convertir_formato_fecha(data['fecha_registro'])

In [107]:
# modificacion del nombre de las claves, con una k_ por delante, por que los campos booblenos (1_1, 2_2, etc) no se pueden trabajar bien
data = {'k_'+key : value for (key, value) in data.items()}
# convertir data(dict) en data
insp = convertir_dict_obj(data, 'insp')

In [108]:
# buscar todas las fotos de un id especifico
fotos_res = buscar_fotos('gallery_57490', insp.k_amigo_id)
# se selecciona solo la data de datos de todo lo retornado
fotos = fotos_res['data']

In [109]:
# crea el template para el reporte
doc = DocxTemplate(ruta + '/templates/tpl_insp_cosechadoras-preventivo.docx')

In [91]:
# descargar lista de fotos, y convertir a inline del template doc
lista_img_inline = descargar_convertir_fotos_inline(fotos, ruta + '/fotos/', doc)

In [110]:
# generar texto contenido del qr
texto_qr = f'''
--UTEA--
--PROGRAMA CAÑA LIMPIA--

Id de Evaluacion: {insp.k_id}
Fecha de Evaluacion: {insp.k_fecha_registro}
Nombre de Canero: {insp.k_canhero.split(' / ')[1]}
Codigo Canero: {insp.k_canhero.split(' / ')[0]}
Maquina Cosechadora: {insp.k_maquina + '-' + insp.k_modelo}
Codigo de Maquina: {insp.k_codigo_maquina}
Calificacion de Evaluacion: {round(insp.k_nota_porcen*100, 2)}%'''

In [111]:
# se genera el qr
ruta_qr = generar_qr(texto_qr, ruta + '/qrs/', insp.k_amigo_id)
# convertir el qr en inline
qr_inline = convertir_img_to_inline(doc, ruta_qr, 30)

In [112]:
# se crea el contexto de datos para pasar el template doc
context = {'insp':insp, 'fotos':lista_img_inline, 'qr':qr_inline}
# renderizar el doc
doc.render(context)
# crea lista con el codigo y nombre del canhero
codigo_nombre = insp.k_canhero.split(' / ')
# crea la ruta y nombre del informe a crear
file_name = codigo_nombre[0] + '_EMPC_' + insp.k_fecha_registro.replace('/','-') + '_' + codigo_nombre[1] + '_' + str(insp.k_id)
# se crea el documento
doc.save(ruta + r'/informes/_' + file_name + '.docx')